In [2]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from all_staff import *

In [3]:
with open(obj_img_dir, 'rb') as f:
    objpoints, imgpoints = pickle.load(f)

In [4]:
def process_image(img):
    undist = undistort_img(img, objpoints, imgpoints)
    binary_img = binary_color(undist, sobel_kernel=3, gray_threshold=(45,255), color_thresold=(110,255))
    binary_warped, Minv = warp_M(binary_img)
    ploty, left_fitx, right_fitx, left_fit, right_fit = slide_windows(binary_warped, nwindows=9, margin=100, minpix=50)
    ploty, left_fitx, right_fitx = acc_frame_to_frame(binary_warped, left_fit, right_fit, margin = 100)
    result = img_region(binary_warped, left_fitx, right_fitx, ploty, Minv, undist)
    return result

In [5]:
white_output = 'vedio/project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("vedio/project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video vedio/project_video_output.mp4
[MoviePy] Writing video vedio/project_video_output.mp4


 11%|█▏        | 142/1261 [01:58<15:52,  1.18it/s]

KeyboardInterrupt: 

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))